In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import VOCDetection
from torch import nn
import time


In [2]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class VOCDataset(Dataset):
    def __init__(self, root_dir, images_dirs, transform=None):
        self.root_dir = root_dir
        self.images_dirs = images_dirs
        self.transform = transform
        self.images = []
        self.labels = []

        # Collect images and labels from specified directories
        for dir_name in images_dirs:
            image_dir = os.path.join(root_dir, dir_name)
            for img_name in os.listdir(image_dir):
                if img_name.endswith('.jpg'):
                    self.images.append(os.path.join(image_dir, img_name))
                    # Placeholder for labels; replace with actual labels extraction
                    # from VOC annotation files (e.g., XML parsing if needed).
                    self.labels.append(0)  # Placeholder, set correct label per image.

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Instantiate the dataset and dataloader
train_images_dirs = ['images/train2012', 'images/train2007', 'images/val2012', 'images/val2007']
train_dataset = VOCDataset(root_dir='./datasets/VOC', images_dirs=train_images_dirs, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)


In [3]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

"""class CustomStudentModel(nn.Module):
    def __init__(self, num_classes=20):
        super(CustomStudentModel, self).__init__()
        
        # Backbone
        self.layer1 = BasicBlock(3, 16, stride=2)
        self.layer2 = BasicBlock(16, 32, stride=2)
        self.layer3 = BasicBlock(32, 64, stride=2)
        self.layer4 = BasicBlock(64, 128, stride=2)
        
        # Neck
        self.neck = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1),
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(64, 32, kernel_size=1)
        )
        
        # Head
        self.head = nn.Conv2d(32, num_classes + 5, kernel_size=1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.neck(x)
        x = self.head(x)
        
        return x

# Initialize model
student_model = CustomStudentModel(num_classes=20)"""


class CustomStudentModel(nn.Module):
    def __init__(self, num_classes=20):
        super(CustomStudentModel, self).__init__()
        
        # Backbone
        self.layer1 = BasicBlock(3, 16, stride=2)
        self.layer2 = BasicBlock(16, 32, stride=2)
        self.layer3 = BasicBlock(32, 64, stride=2)
        self.layer4 = BasicBlock(64, 128, stride=2)
        
        # Neck
        self.neck = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1),
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(64, 32, kernel_size=1)
        )
        
        # Head
        self.head = nn.Conv2d(32, num_classes, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling to reduce dimensions

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.neck(x)
        x = self.head(x)
        x = self.pool(x)  # Shape becomes [batch_size, num_classes, 1, 1]
        return x.view(x.size(0), -1)  # Flatten to [batch_size, num_classes]

# Initialize and test model
student_model = CustomStudentModel(num_classes=20)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.001)


In [6]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# student_model.to(device)
# num_epochs = 5

# for epoch in range(num_epochs):
#     student_model.train()
#     running_loss = 0.0
    
#     for i, (images, labels) in enumerate(train_loader):
#         images = images.to(device)
#         labels = labels.to(device)
        
#         # Forward pass
#         outputs = student_model(images)
#         loss = criterion(outputs, labels)
        
#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
#         if (i + 1) % 10 == 0:
#             print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
    
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# print("Training completed.")


In [7]:
# class CustomStudentModel(nn.Module):
#     def __init__(self, num_classes=20):
#         super(CustomStudentModel, self).__init__()
        
#         # Backbone
#         self.layer1 = BasicBlock(3, 16, stride=2)
#         self.layer2 = BasicBlock(16, 32, stride=2)
#         self.layer3 = BasicBlock(32, 64, stride=2)
#         self.layer4 = BasicBlock(64, 128, stride=2)
        
#         # Neck
#         self.neck = nn.Sequential(
#             nn.Conv2d(128, 64, kernel_size=1),
#             nn.Upsample(scale_factor=2, mode="nearest"),
#             nn.Conv2d(64, 32, kernel_size=1)
#         )
        
#         # Head
#         self.head = nn.Conv2d(32, num_classes, kernel_size=1)
#         self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling to reduce dimensions

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
        
#         x = self.neck(x)
#         x = self.head(x)
#         x = self.pool(x)  # Shape becomes [batch_size, num_classes, 1, 1]
#         return x.view(x.size(0), -1)  # Flatten to [batch_size, num_classes]

# # Initialize and test model
# student_model = CustomStudentModel(num_classes=20)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
student_model.to(device)
num_epochs = 5


for epoch in range(num_epochs):
    student_model.train()
    running_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = student_model(images)  # Shape [batch_size, num_classes]
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training completed.")
